## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import sys
sys.path.insert(0, '..')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
#Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2022-03-26 14:02:51,-23.1203,-134.9692,78.76,74,6,14.25,clear sky
1,1,Douentza,ML,2022-03-26 14:02:52,14.9951,-2.9517,101.17,19,11,8.01,few clouds
2,2,Port Elizabeth,ZA,2022-03-26 14:02:53,-33.9180,25.5701,69.31,79,74,3.00,broken clouds
3,3,Bredasdorp,ZA,2022-03-26 14:02:53,-34.5322,20.0403,70.63,58,60,18.21,broken clouds
4,4,Kapaa,US,2022-03-26 14:02:19,22.0752,-159.3190,66.18,66,99,13.09,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
print(f"Max temperature is: {max_temp}")
print(f"Min temperature is: {min_temp}")

What is the maximum temperature you would like for your trip? 85
What is the minimum temperature you would like for your trip? 70
Max temperature is: 85.0
Min temperature is: 70.0


In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2022-03-26 14:02:51,-23.1203,-134.9692,78.76,74,6,14.25,clear sky
3,3,Bredasdorp,ZA,2022-03-26 14:02:53,-34.5322,20.0403,70.63,58,60,18.21,broken clouds
7,7,Carnarvon,AU,2022-03-26 14:02:54,-24.8667,113.6333,77.45,70,60,12.68,broken clouds
13,13,Victoria,HK,2022-03-26 14:00:01,22.2855,114.1577,80.65,88,17,10.00,few clouds
20,20,Latung,PH,2022-03-26 14:02:58,5.5000,120.8833,83.16,77,94,11.83,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...
660,660,Eydhafushi,MV,2022-03-26 14:19:00,5.1033,73.0708,82.51,64,0,5.84,clear sky
664,664,Pandan,PH,2022-03-26 14:19:02,14.0453,124.1694,79.39,84,70,6.49,broken clouds
665,665,Montepuez,MZ,2022-03-26 14:19:02,-13.1256,38.9997,81.55,79,77,1.59,light rain
667,667,Bac Lieu,VN,2022-03-26 14:14:40,9.2850,105.7244,83.75,72,0,7.72,clear sky


In [5]:
#Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
#Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
#Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

#Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.76,clear sky,-23.1203,-134.9692,
3,Bredasdorp,ZA,70.63,broken clouds,-34.5322,20.0403,
7,Carnarvon,AU,77.45,broken clouds,-24.8667,113.6333,
13,Victoria,HK,80.65,few clouds,22.2855,114.1577,
20,Latung,PH,83.16,overcast clouds,5.5000,120.8833,
21,Sao Filipe,CV,70.68,clear sky,14.8961,-24.4956,
33,Dingle,PH,78.98,few clouds,10.9995,122.6711,
34,Puerto Baquerizo Moreno,EC,78.60,overcast clouds,-0.9000,-89.6000,
35,Atuona,PF,78.73,overcast clouds,-9.8000,-139.0333,
39,San Rafael,AR,70.88,clear sky,-34.6177,-68.3301,


In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df
clean_hotel_df.isna().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.76,clear sky,-23.1203,-134.9692,People ThankYou
3,Bredasdorp,ZA,70.63,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor
7,Carnarvon,AU,77.45,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
13,Victoria,HK,80.65,few clouds,22.2855,114.1577,Mini Hotel Central
21,Sao Filipe,CV,70.68,clear sky,14.8961,-24.4956,Hotel Xaguate
...,...,...,...,...,...,...,...
660,Eydhafushi,MV,82.51,clear sky,5.1033,73.0708,Soneva Fushi
664,Pandan,PH,79.39,broken clouds,14.0453,124.1694,Bonifacio Lodge
665,Montepuez,MZ,81.55,light rain,-13.1256,38.9997,PembaAparthotel
667,Bac Lieu,VN,83.75,clear sky,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu


In [11]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search//WeatherPy_Vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# get data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# get latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# Add a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))